## Notebook used for exploring WebquestionsSP dataset and saving data in proper format

To use, download WebQSP zip file from https://www.microsoft.com/en-us/download/details.aspx?id=52763 and unzip in the CL4Code folder.

In [19]:
import json

%cd ~/CL4Code/WebQSP/data


fp = open("WebQSP.train.json", "r")
dataset = json.load(fp)

print(dataset.keys())

/data/lily/nos6/CL4Code/WebQSP/data
dict_keys(['Version', 'FreebaseVersion', 'Questions'])


In [23]:
print(dataset['Questions'][0].keys())
print(dataset['Questions'][0])

dict_keys(['QuestionId', 'RawQuestion', 'ProcessedQuestion', 'Parses'])
{'QuestionId': 'WebQTrn-0', 'RawQuestion': 'what is the name of justin bieber brother?', 'ProcessedQuestion': 'what is the name of justin bieber brother', 'Parses': [{'ParseId': 'WebQTrn-0.P0', 'AnnotatorId': 1, 'AnnotatorComment': {'ParseQuality': 'Complete', 'QuestionQuality': 'Good', 'Confidence': 'Normal', 'FreeFormComment': 'First-round parse verification'}, 'Sparql': "PREFIX ns: <http://rdf.freebase.com/ns/>\nSELECT DISTINCT ?x\nWHERE {\nFILTER (?x != ns:m.06w2sn5)\nFILTER (!isLiteral(?x) OR lang(?x) = '' OR langMatches(lang(?x), 'en'))\nns:m.06w2sn5 ns:people.person.sibling_s ?y .\n?y ns:people.sibling_relationship.sibling ?x .\n?x ns:people.person.gender ns:m.05zppz .\n}\n", 'PotentialTopicEntityMention': 'justin bieber', 'TopicEntityName': 'Justin Bieber', 'TopicEntityMid': 'm.06w2sn5', 'InferentialChain': ['people.person.sibling_s', 'people.sibling_relationship.sibling'], 'Constraints': [{'Operator': 'Equ

In [38]:
print(dataset['Questions'][0]['Parses'])
for i in range(0, len(dataset['Questions'])):
    if len(dataset['Questions'][i]['Parses']) != 1:
        print(dataset['Questions'][i]['Parses'][1])
        break
        
print(dataset['Questions'][0]['Parses'][0].keys())

[{'ParseId': 'WebQTrn-0.P0', 'AnnotatorId': 1, 'AnnotatorComment': {'ParseQuality': 'Complete', 'QuestionQuality': 'Good', 'Confidence': 'Normal', 'FreeFormComment': 'First-round parse verification'}, 'Sparql': "PREFIX ns: <http://rdf.freebase.com/ns/>\nSELECT DISTINCT ?x\nWHERE {\nFILTER (?x != ns:m.06w2sn5)\nFILTER (!isLiteral(?x) OR lang(?x) = '' OR langMatches(lang(?x), 'en'))\nns:m.06w2sn5 ns:people.person.sibling_s ?y .\n?y ns:people.sibling_relationship.sibling ?x .\n?x ns:people.person.gender ns:m.05zppz .\n}\n", 'PotentialTopicEntityMention': 'justin bieber', 'TopicEntityName': 'Justin Bieber', 'TopicEntityMid': 'm.06w2sn5', 'InferentialChain': ['people.person.sibling_s', 'people.sibling_relationship.sibling'], 'Constraints': [{'Operator': 'Equal', 'ArgumentType': 'Entity', 'Argument': 'm.05zppz', 'EntityName': 'Male', 'SourceNodeIndex': 1, 'NodePredicate': 'people.person.gender', 'ValueType': 'String'}], 'Time': None, 'Order': None, 'Answers': [{'AnswerType': 'Entity', 'Answe

In [61]:
def load_all_items(dataset):
    instance_list = []

    for item in dataset['Questions']:
        
        instance = {}
        instance['question'] = item['RawQuestion']
        instance['task_id'] = item['QuestionId']
        instance['dsl_code'] = []
        instance['answer'] = []
        
        # add all parses and answers to the instance
        if len(item['Parses']) > 0:
            
            for parse in item['Parses']:
                # matching eval script provided by Microsoft, filter by good quality items
                if parse['AnnotatorComment']['QuestionQuality'] == 'Good' and parse['AnnotatorComment']['ParseQuality'] == 'Complete':
                    instance['dsl_code'].append(parse['Sparql'])
                    
                    for answer in parse['Answers']:
                        if answer['EntityName'] not in instance['answer']:
                            instance['answer'].append(answer['EntityName'])
                        
                    

            instance_list.append(instance)

        else:
            instance = None
    
    return instance_list

instance_list = load_all_items(dataset)

with open('../../cl/data/webqsp_train.json', 'w+') as f: 
    json.dump(instance_list, f)



In [62]:
fp = open("WebQSP.test.json", "r")
test_dataset = json.load(fp)

test_list = load_all_items(test_dataset)

with open('../../cl/data/webqsp_test.json', 'w+') as f: 
    json.dump(test_list, f)